In [1]:
import tensorflow as tf
from tensorflow.keras.mixed_precision import experimental as mixed_precision
# mode of computation - use combination of float16 and float32 precisions for computations

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [2]:
import importlib
import utils.model
import utils.architectures
importlib.reload(utils.model)
importlib.reload(utils.architectures)

<module 'utils.architectures' from '/home/strv/Projects/mask2face/utils/architectures.py'>

In [3]:
from utils.architectures import UNet
from utils.model import Mask2FaceModel

filters = (128, 128, 128, 128, 128, 64, 64, 64)
# filters = (128, 128, 64, 64, 32, 32, 16, 8) # Good with Resnet
filters = (256, 256, 128, 128, 64, 64, 32, 16) # Filters for models/unet_resnet_deep.h5 model
# filters = (128, 64, 32, 16)

model = Mask2FaceModel(arch=UNet.RESNET, filters=filters)
model.summary()

UNet.RESNET (256, 256) (256, 256, 128, 128, 64, 64, 32, 16)
Using UNet Resnet model
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 256 7168        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 256 590080      conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 256 590080      conv2d_1[0][0]                   
__________

In [4]:
model.train(epochs=40, batch_size=12, loss_function='ssim_loss', predict_difference=True)

Epoch 1/40
249/249 [==============================] - 318s 1s/step - loss: 0.0517 - acc: 0.6915 - recall: 0.6533 - precision: 1.0000 - iou: 0.3445 - val_loss: 0.0280 - val_acc: 0.5229 - val_recall: 0.2087 - val_precision: 1.0000 - val_iou: 0.3498 - lr: 1.0000e-04

In [5]:
# model.load_model('model_epochs-40_batch-12_loss-ssim_loss_20210309_16_57_33.h5')
model.predict_batch(predict_difference=True)

In [5]:

model.predict('data/IMG_9667.png')

In [8]:
import cv2
import numpy as np
def predict(img_path, predict_difference=True):
    x = Mask2FaceModel.read_image(img_path)
    y_pred = model.model.predict(np.expand_dims(x, axis=0))
    if predict_difference:
        y_pred = (y_pred * 2) - 1
        y_pred = np.clip(x - y_pred.squeeze(axis=0), 0.0, 1.0)
    else:
        y_pred = y_pred.squeeze(axis=0)
    h, w, _ = x.shape
    white_line = np.ones((h, 10, 3)) * 255.0

    all_images = [
        x * 255.0, white_line,
        y_pred.squeeze(axis=0) * 255.0
    ]
    image = np.concatenate(all_images, axis=1)
    cv2.imwrite(img_path.split('.')[0]+'_unmasked.png', image)

In [9]:
predict('data/IMG_9666.png')

ValueError: cannot select an axis to squeeze out which has size not equal to one

In [10]:
import os
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf

INPUT_TENSOR_NAME = 'ImageTensor:0'
OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
INPUT_SIZE = 513

with tf.io.gfile.GFile('frozen_inference_graph_face.pb', "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    g_in = tf.import_graph_def(graph_def, name="")
sess = tf.compat.v1.Session(graph=g_in)


def run(image):
    width, height = image.size
    resize_ratio = 1.0 * INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = sess.run(
        OUTPUT_TENSOR_NAME,
        feed_dict={INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map

input_image = Image.open('data/IMG_9666.png')
resized_im, seg_map = run(input_image)
fig = plt.figure()
fig.add_subplot(1, 2, 1)
plt.imshow(resized_im)
fig.add_subplot(1, 2, 2)
plt.imshow(np.ma.masked_equal(seg_map, 0))

TypeError: Cannot interpret feed_dict key as Tensor: The name 'ImageTensor:0' refers to a Tensor which does not exist. The operation, 'ImageTensor', does not exist in the graph.

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from utils.facenet_model import *

p0 = Input((256,256,3))



base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False, 
                                               pooling='avg',
                                               weights='imagenet')
# model = InceptionResNetV1(input_shape=(256, 256, 3), weights_path='utils/facenet_keras_weights.h5')


y = base_model(p0)

# y = RepeatVector(32*32)(y)
# y = Reshape((32,32,1280))(y)
m = Model(p0, y)
# m.summary()
# outputs = Conv2D(3, (1, 1), padding="same", activation="sigmoid")(x)

# base_model.trainable = False
# for layer in base_model.layers:
#     layer.trainable = False
    
# m = Model(p0, outputs)
m.summary()
# base_model.trainable = False
# base_model.layers[0].trainable

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Model) (None, 1280)              2257984   
Total params: 2,257,984
Trainable params: 2,223,872
Non-trainable params: 34,112
_________________________________________________________________


In [1]:
from utils.facenet_model import *
model = InceptionResNetV1(input_shape=(256, 256, 3), weights_path='utils/facenet_keras_weights.h5')
model.summary

In [1]:
from utils.architectures import *
m = UNet_segmentation()

In [3]:
m.model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 16) 2320        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 16) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [1]:
import importlib
from utils.configuration import Configuration
from utils.data_generator import DataGenerator

In [2]:
importlib.reload(utils.configuration)
importlib.reload(utils.data_generator)

NameError: name 'utils' is not defined

In [3]:
import random
c = Configuration()
dg = DataGenerator(c)
dg.generate_images()